In [1]:
import re
import json
import numpy as np
import pandas as pd

In [2]:
def get_type(extend_map, label):
    for key, val in extend_map.items():
        for v in val:
            if v == label:
                return key


def get_count_matrix(eve_extend_map, merged_golds, test):
    new_eve_extend_map = {k:[] for k in eve_extend_map.keys()}
    for k in new_eve_extend_map.keys():
        # new_eve_extend_map[k] = [k] + eve_extend_map[k]
        new_eve_extend_map[k] = eve_extend_map[k]
    value = [c for v in new_eve_extend_map.values() for c in v]
    dict = {k: [] for k in value}
    dict_count = {k: [] for k in value}

    for item in test:
        lab = item[0][1]
        dict[lab].append(item)

    for k,v in dict.items():   
        cur_extend_eve = k
        cur_list = v
        for item in cur_list:
            count_correct = 0
            count_wrong_from_gold = 0
            count_wrong_from_pred = 0

            cur_gold_eve = get_type(new_eve_extend_map, cur_extend_eve)
            cur_idx = item[0][0]
            cur_merged_golds = merged_golds[cur_idx]
            if type(cur_merged_golds) == tuple:
                cur_golds = [cur_merged_golds]
            else:
                cur_golds = [gold for gold in cur_merged_golds if gold[1] == cur_gold_eve]
            cur_preds = [pred for pred in item] 

            preds_length = len(cur_preds)
            golds_length = len(cur_golds)

            if golds_length != 0:
                for pred in cur_preds:
                    for cmp_gold in cur_golds:
                        if pred[2] == '':
                            preds_length -= 1
                            break
                        elif pred[2] in cmp_gold[2] or cmp_gold[2] in pred[2]:
                            count_correct += 1
                            break
            count_correct = min(count_correct, golds_length)
            count_wrong_from_gold = golds_length - count_correct
            count_wrong_from_pred = preds_length - count_correct
            dict_count[cur_extend_eve].append((count_correct, count_wrong_from_gold, count_wrong_from_pred))

    matrix = [v for _,v in dict_count.items()]

    return matrix

In [3]:
def word_level(eve_extend_map, matrix):
    new_eve_extend_map = {k:[] for k in eve_extend_map.keys()}
    for k in new_eve_extend_map.keys():
        # new_eve_extend_map[k] = [k] + eve_extend_map[k]
        new_eve_extend_map[k] = eve_extend_map[k]
    value = [c for v in new_eve_extend_map.values() for c in v]
    word_count_dict = {k:[] for k in value}

    tmp_list = []

    for idx, row in enumerate(matrix):
        correct_sum = 0
        wrong_from_gold_sum = 0
        wrong_from_pred_sum = 0
        for tuple in row:
            correct_sum += tuple[0]
            wrong_from_gold_sum += tuple[1]
            wrong_from_pred_sum += tuple[2]
        if correct_sum + wrong_from_pred_sum == 0:
            P = 0.0
        else:
            P = correct_sum / (correct_sum + wrong_from_pred_sum)
        if correct_sum + wrong_from_gold_sum == 0:
            R = 0.0
        else:
            R = correct_sum / (correct_sum + wrong_from_gold_sum)
        
        if P + R == 0:
            F1 = 0.0
        else:
            F1 = 2 * P * R / (P + R)
        P_1 = 1 - P
        R_1 = 1 - R
        tmp_list.append((P, R, P_1, R_1, F1, correct_sum, wrong_from_gold_sum, wrong_from_pred_sum))


    for idx,k in enumerate(word_count_dict.keys()):
        word_count_dict[k] = tmp_list[idx]

    return word_count_dict


def word_dict2execl(word_count_dict, eve_extend_map, human_sim, mode):
    new_eve_extend_map = {k:[] for k in eve_extend_map.keys()}
    for k in new_eve_extend_map.keys():
        # new_eve_extend_map[k] = [k] + eve_extend_map[k]
        new_eve_extend_map[k] = eve_extend_map[k]

    df = pd.DataFrame.from_dict(word_count_dict, orient='index').reset_index()
    df.columns = ['扩展事件词','P','R','1-P','1-R','F1','正确个数','golds中错误个数','preds中错误个数']
    extend_eve_list = df['扩展事件词'].tolist()

    gold_eve_list = [get_type(new_eve_extend_map,eve) for eve in extend_eve_list]
    df['事件类型'] = gold_eve_list

    cols = list(df.columns)
    cols.insert(0, cols.pop(cols.index('事件类型')))
    df = df.loc[:, cols]
    
    df.insert(loc=2, column='是否语义相关', value=human_sim)

    df.to_excel(f'./results/word_count_dict_{mode}.xlsx',index=False)

    return df



In [4]:
import pandas as pd

def split_sheet(groupby_col, mode):  
    df = pd.read_excel(f'./results/word_count_dict_{mode}.xlsx')
    df_grouped = df.groupby(groupby_col)
    type_list = list(df[groupby_col].unique())

    with pd.ExcelWriter(f'./results/word_count_dict_{mode}_中间处理表.xlsx', engine='xlsxwriter') as writer:
        for type in type_list:
            sub_df = df_grouped.get_group(type).copy().reset_index(drop=True)
            if r'/' in type:
                type = type.replace(r'/', '')
            sub_df.to_excel(writer, sheet_name=type, index=False)
        

DuEE

In [8]:
with open('./data/duee/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/duee/event_extend_map.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

with open('./data/duee/merged_golds.txt', 'r', encoding='utf-8') as f:
    merged_golds = [eval(line) for line in f]

with open('./results/test_zh.txt', 'r', encoding='utf-8') as f:
    test = [eval(line) for line in f]  


matrix = get_count_matrix(eve_extend_map, merged_golds, test)
word_count_dict = word_level(eve_extend_map, matrix)
df_word = word_dict2execl(word_count_dict, eve_extend_map, mode='zh')

CASIE

In [11]:
with open('./data/casie/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/casie/event_extend_map.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

with open('./data/casie/golds.txt', 'r', encoding='utf-8') as f:
    merged_golds = [eval(line) for line in f]

with open('./results/test_en.txt', 'r', encoding='utf-8') as f:
    test = [eval(line) for line in f]

matrix = get_count_matrix(eve_extend_map, merged_golds, test)
word_count_dict = word_level(eve_extend_map, matrix)
df_word = word_dict2execl(word_count_dict, eve_extend_map, mode='en')

ChatGLM-DuEE

In [5]:
with open('./data/duee/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/duee/final_event_extend_map_chatglm_6B_zh.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

with open('./data/duee/merged_golds.txt', 'r', encoding='utf-8') as f:
    merged_golds = [eval(line) for line in f]

with open('./results/test_chatglm_6B_zh.txt', 'r', encoding='utf-8') as f:
    test = [eval(line) for line in f]  


matrix = get_count_matrix(eve_extend_map, merged_golds, test)
word_count_dict = word_level(eve_extend_map, matrix)
df_word = word_dict2execl(word_count_dict, eve_extend_map, mode='chatglm_6B_zh')

ChatGLM-CASIE

In [7]:
with open('./data/casie/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/casie/final_event_extend_map_chatglm_6B_en.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

with open('./data/casie/golds.txt', 'r', encoding='utf-8') as f:
    merged_golds = [eval(line) for line in f]

with open('./results/test_chatglm_6B_en.txt', 'r', encoding='utf-8') as f:
    test = [eval(line) for line in f]

matrix = get_count_matrix(eve_extend_map, merged_golds, test)
word_count_dict = word_level(eve_extend_map, matrix)
df_word = word_dict2execl(word_count_dict, eve_extend_map, mode='chatglm_6B_en')

Baichuan2-DuEE

In [14]:
with open('./data/duee/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/duee/final_event_extend_map_baichuan2_13B_zh.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

with open('./data/duee/merged_golds.txt', 'r', encoding='utf-8') as f:
    merged_golds = [eval(line) for line in f]

with open('./results/test_baichuan2_13B_zh.txt', 'r', encoding='utf-8') as f:
    test = [eval(line) for line in f]  


with open('./data/duee/event_sim_human_baichuan2_13B_zh.json','r',encoding='utf-8') as f:
    sim_en = json.load(f)
res_dict = {k:{} for k in eve_extend_map.keys()}
for key in eve_extend_map.keys():
    for value in sim_en[key].keys():
        if value in eve_extend_map[key]:
            res_dict[key][value] = sim_en[key][value]

with open('./data/duee/final_event_sim_human_baichuan2_13B_zh.json','w',encoding='utf-8') as f:
    json.dump(res_dict,f,indent=4,ensure_ascii=False)

human_sim = []
for key in res_dict.keys():
    for k,v in res_dict[key].items():
        human_sim.append(v)
        
matrix = get_count_matrix(eve_extend_map, merged_golds, test)
word_count_dict = word_level(eve_extend_map, matrix)
df_word = word_dict2execl(word_count_dict, eve_extend_map, human_sim, 'baichuan2_13B_zh')

split_sheet('事件类型','baichuan2_13B_zh')

Baichuan2-CASIE

In [16]:
with open('./data/casie/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/casie/final_event_extend_map_baichuan2_13B_en.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

with open('./data/casie/golds.txt', 'r', encoding='utf-8') as f:
    merged_golds = [eval(line) for line in f]

with open('./results/test_baichuan2_13B_en.txt', 'r', encoding='utf-8') as f:
    test = [eval(line) for line in f]

with open('./data/casie/event_sim_human_baichuan2_13B_en.json','r',encoding='utf-8') as f:
    sim_en = json.load(f)
res_dict = {k:{} for k in eve_extend_map.keys()}
for key in eve_extend_map.keys():
    for value in sim_en[key].keys():
        if value in eve_extend_map[key]:
            res_dict[key][value] = sim_en[key][value]

with open('./data/duee/final_event_sim_human_baichuan2_13B_en.json','w',encoding='utf-8') as f:
    json.dump(res_dict,f,indent=4,ensure_ascii=False)

human_sim = []
for key in res_dict.keys():
    for k,v in res_dict[key].items():
        human_sim.append(v)
        
matrix = get_count_matrix(eve_extend_map, merged_golds, test)
word_count_dict = word_level(eve_extend_map, matrix)
df_word = word_dict2execl(word_count_dict, eve_extend_map, human_sim ,'baichuan2_13B_en')

split_sheet('事件类型','baichuan2_13B_en')

Alpaca

In [5]:
with open('./data/duee/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/duee/final_event_extend_map_alpaca_33B_zh.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

with open('./data/duee/merged_golds.txt', 'r', encoding='utf-8') as f:
    merged_golds = [eval(line) for line in f]

with open('./results/test_alpaca_33B_zh.txt', 'r', encoding='utf-8') as f:
    test = [eval(line) for line in f]  


with open('./data/duee/event_sim_human_alpaca_33B_zh.json','r',encoding='utf-8') as f:
    sim_en = json.load(f)
res_dict = {k:{} for k in eve_extend_map.keys()}
for key in eve_extend_map.keys():
    for value in sim_en[key].keys():
        if value in eve_extend_map[key]:
            res_dict[key][value] = sim_en[key][value]

with open('./data/duee/final_event_sim_human_alpaca_33B_zh.json','w',encoding='utf-8') as f:
    json.dump(res_dict,f,indent=4,ensure_ascii=False)

human_sim = []
for key in res_dict.keys():
    for k,v in res_dict[key].items():
        human_sim.append(v)
        
matrix = get_count_matrix(eve_extend_map, merged_golds, test)
word_count_dict = word_level(eve_extend_map, matrix)
df_word = word_dict2execl(word_count_dict, eve_extend_map, human_sim, 'alpaca_33B_zh')

split_sheet('事件类型','alpaca_33B_zh')

In [6]:
with open('./data/casie/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/casie/final_event_extend_map_alpaca_33B_en.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

with open('./data/casie/golds.txt', 'r', encoding='utf-8') as f:
    merged_golds = [eval(line) for line in f]

with open('./results/test_alpaca_33B_en.txt', 'r', encoding='utf-8') as f:
    test = [eval(line) for line in f]

with open('./data/casie/event_sim_human_alpaca_33B_en.json','r',encoding='utf-8') as f:
    sim_en = json.load(f)
res_dict = {k:{} for k in eve_extend_map.keys()}
for key in eve_extend_map.keys():
    for value in sim_en[key].keys():
        if value in eve_extend_map[key]:
            res_dict[key][value] = sim_en[key][value]

with open('./data/duee/final_event_sim_human_alpaca_33B_en.json','w',encoding='utf-8') as f:
    json.dump(res_dict,f,indent=4,ensure_ascii=False)

human_sim = []
for key in res_dict.keys():
    for k,v in res_dict[key].items():
        human_sim.append(v)
        
matrix = get_count_matrix(eve_extend_map, merged_golds, test)
word_count_dict = word_level(eve_extend_map, matrix)
df_word = word_dict2execl(word_count_dict, eve_extend_map, human_sim ,'alpaca_33B_en')

split_sheet('事件类型','alpaca_33B_en')